In [89]:
import google.auth
from google.oauth2 import service_account
from google.cloud import bigquery
from google.cloud import bigquery_storage_v1beta1

import pandas as pd
import numpy as np

from sklearn.preprocessing import Normalizer

%matplotlib inline

In [32]:
credentials = service_account.Credentials.from_service_account_file('DSN-GAS360-SA.json')

In [3]:
project_id = 'dsn-gas360'

In [4]:
bqclient = bigquery.Client(
    credentials = credentials,
    project = project_id
)

In [5]:
bqstorageclient = bigquery_storage_v1beta1.BigQueryStorageClient(credentials = credentials)

In [1]:
query = """
    SELECT 
      fullVisitorId, visitId, date, socialEngagementType,
      totals.bounces, totals.hits, totals.newVisits, totals.sessionQualityDim, totals.timeOnScreen, totals.timeOnSite,
      totals.transactions, totals.totalTransactionRevenue, totals.visits,
      trafficSource.source, trafficSource.referralPath, trafficSource.medium, trafficSource.keyword, 
      device.browser, device.deviceCategory, device.operatingSystem, device.language,
      geoNetwork.continent, geoNetwork.country, geoNetwork.region, geoNetwork.metro, geoNetwork.city
    FROM
      `bigquery-public-data.google_analytics_sample.ga_sessions_*`
    WHERE
      _TABLE_SUFFIX BETWEEN '20160801' AND '20170831'
    """

In [7]:
def execute_big_query(query):
    df = (bqclient.query(query).result().to_dataframe(bqstorage_client = bqstorageclient))
    display(df.head())
    return df

In [8]:
df = execute_big_query(query)

,fullVisitorId,date,bounces,newVisits,sessionQualityDim,timeOnScreen,timeOnSite,transactions,totalTransactionRevenue,visits,...,keyword,browser,deviceCategory,operatingSystem,language,continent,country,region,metro,city
0,2558542945282276924,20160901,1.0,NaN,NaN,NaN,NaN,NaN,NaN,1,...,(not provided),Chrome,mobile,Android,not available in demo dataset,Oceania,Australia,New South Wales,(not set),Sydney
1,3559409406108279851,20160901,1.0,1.0,NaN,NaN,NaN,NaN,NaN,1,...,(not provided),Chrome,desktop,Windows,not available in demo dataset,Europe,France,Ile-de-France,(not set),Paris
2,1236280516298011032,20160901,1.0,1.0,NaN,NaN,NaN,NaN,NaN,1,...,None,Chrome,desktop,Macintosh,not available in demo dataset,Asia,China,not available in demo dataset,not available in demo dataset,not available in demo dataset
3,9717817368523884345,20160901,1.0,1.0,NaN,NaN,NaN,NaN,NaN,1,...,(not provided),Chrome,desktop,Windows,not available in demo dataset,Europe,Belgium,(not set),(not set),(not set)
4,9422257354738249512,20160901,1.0,NaN,NaN,NaN,NaN,NaN,NaN,1,...,(not provided),Chrome,desktop,Macintosh,not available in demo dataset,Americas,United States,New York,New York NY,New York


In [9]:
df.to_csv('ga360_source_query.csv', index = False, header = True)